In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#data1=pd.read_csv('bs140513_032310.csv')
data1=pd.read_csv('D:\\DataScienceCompetitions\\AI4Hackathon\\synthetic-data-from-a-financial-payment-system\\bs140513_032310.csv')

In [3]:
data1.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [4]:
#zipcodeOri and zipMerchant doesn't have any predictive power, zero variance(unique value)
data1=data1.drop(columns=['zipcodeOri', 'zipMerchant'])

In [5]:
data1.shape

(594643, 8)

In [6]:
df=data1.groupby('customer').agg({'fraud':'sum','category':'count' }).reset_index().\
rename(columns = {'category':'num_transactions', 'fraud':'num_frauds'})

In [7]:
df['cus_ratio_fraud']=df['num_frauds']/df['num_transactions']

In [8]:
data1=pd.merge(data1, df[['customer', 'cus_ratio_fraud']], on='customer')

In [9]:
df=data1.groupby('merchant').agg({'fraud':'sum','category':'count' }).reset_index().\
rename(columns = {'category':'num_transactions', 'fraud':'num_frauds'})

In [10]:
df['mer_ratio_fraud']=df['num_frauds']/df['num_transactions']

In [11]:
data1=pd.merge(data1, df[['merchant', 'mer_ratio_fraud']], on='merchant')

In [12]:
#dropping customer id
data1=data1.drop(columns=['customer','merchant'])

In [13]:
data1.head()

,step,age,gender,category,amount,fraud,cus_ratio_fraud,mer_ratio_fraud
0,0,'4','M','es_transportation',4.55,0,0.0,0.0
1,2,'4','M','es_transportation',37.21,0,0.0,0.0
2,3,'4','M','es_transportation',31.63,0,0.0,0.0
3,4,'4','M','es_transportation',35.86,0,0.0,0.0
4,6,'4','M','es_transportation',39.58,0,0.0,0.0


In [14]:
x=list(data1.columns)

In [15]:
x.remove('fraud')

In [16]:
x

['step',
 'age',
 'gender',
 'category',
 'amount',
 'cus_ratio_fraud',
 'mer_ratio_fraud']

In [17]:
X=data1.loc[:,x].values
y=data1.iloc[:, 5].values

In [18]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [19]:
labelencoder_X1=LabelEncoder()
X[:,1]=labelencoder_X1.fit_transform(X[:,1])

labelencoder_X2=LabelEncoder()
X[:,2]=labelencoder_X2.fit_transform(X[:,2])

labelencoder_X3=LabelEncoder()
X[:,3]=labelencoder_X3.fit_transform(X[:,3])

In [20]:
# Encoding categorical data
onehotencoder = OneHotEncoder(categorical_features=[1,2,3])
X = onehotencoder.fit_transform(X).toarray()

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



# DecisionTree

In [25]:
from sklearn.tree import DecisionTreeClassifier
classifier_DT=DecisionTreeClassifier(criterion='entropy', random_state=0)
classifier_DT.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [26]:
y_pred=classifier_DT.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117194,    299],
       [   279,   1157]])

In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9015827369742907

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.79      0.81      0.80      1436

avg / total       1.00      1.00      1.00    118929



# Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF=RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier_RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [31]:
y_pred=classifier_RF.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117377,    116],
       [   262,   1174]])

In [33]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9082807268290528

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.91      0.82      0.86      1436

avg / total       1.00      1.00      1.00    118929



# SVM(Kernal=Poly)

In [35]:
from sklearn.svm import SVC
classifier_pl=SVC(kernel='poly', random_state=0)
classifier_pl.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [36]:
y_pred=classifier_pl.predict(X_test)

In [37]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117385,    108],
       [   333,   1103]])

In [38]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8835933229424656

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.91      0.77      0.83      1436

avg / total       1.00      1.00      1.00    118929



# ANN

In [40]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [41]:
classifier = Sequential()

In [42]:
len(X_train[1])

31

In [43]:
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 31))
#classifier.add(Dropout(p = 0.2))
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
#classifier.add(Dropout(p = 0.2))
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
#classifier.add(Dropout(p = 0.2))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [44]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                512       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
_________________________________________________________________


In [45]:
classifier.fit(X_train, y_train, batch_size = 100, epochs = 30)

Epoch 1/30
475714/475714 [==============================] - 25s 53us/step - loss: 0.0205 - acc: 0.9948
Epoch 2/30
475714/475714 [==============================] - 24s 50us/step - loss: 0.0101 - acc: 0.9962
Epoch 3/30
475714/475714 [==============================] - 27s 57us/step - loss: 0.0096 - acc: 0.9963
Epoch 4/30
475714/475714 [==============================] - 24s 51us/step - loss: 0.0094 - acc: 0.9964
Epoch 5/30
475714/475714 [==============================] - 24s 51us/step - loss: 0.0092 - acc: 0.9965
Epoch 6/30
475714/475714 [==============================] - 25s 52us/step - loss: 0.0091 - acc: 0.9964
Epoch 7/30
475714/475714 [==============================] - 25s 52us/step - loss: 0.0089 - acc: 0.9965
Epoch 8/30
475714/475714 [==============================] - 28s 58us/step - loss: 0.0089 - acc: 0.9965
Epoch 9/30
475714/475714 [==============================] - 24s 50us/step - loss: 0.0087 - acc: 0.9965
Epoch 10/30
475714/475714 [==============================] - 21s 44us/ste

In [46]:
y_pred=classifier.predict(X_test)

In [54]:
y_pred=[np.round(x[0]) for x in y_pred]

In [55]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117382,    111],
       [   303,   1133]])

In [56]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8940262386757017

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.91      0.79      0.85      1436

avg / total       1.00      1.00      1.00    118929



# XGBoost

In [23]:
from xgboost import XGBClassifier
classifier_xg = XGBClassifier()
classifier_xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [24]:
y_pred=classifier_xg.predict(X_test)

C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117375,    118],
       [   287,   1149]], dtype=int64)

In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8995674803076634

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.91      0.80      0.85      1436

avg / total       1.00      1.00      1.00    118929



# Cross Validation

In [28]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_xg, X=X_train, y=y_train, cv =10)
accuracies.mean()


C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\labe

0.9965756750503578

In [29]:
accuracies

array([0.99646851, 0.99621626, 0.99663668, 0.99630034, 0.99669967,
       0.99676273, 0.99655252, 0.99632129, 0.99669967, 0.99709907])

# SVM Kernal=Linear

In [58]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear', random_state=0)
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [59]:
y_pred=classifier.predict(X_test)

In [60]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117370,    123],
       [   327,   1109]])

In [61]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8856186258426301

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.90      0.77      0.83      1436

avg / total       1.00      1.00      1.00    118929



# Logistic Regression

In [63]:
from sklearn.linear_model import LogisticRegression
classifier_lr=LogisticRegression(random_state=0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
y_pred=classifier_lr.predict(X_test)

In [65]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117364,    129],
       [   346,   1090]])

In [66]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8789774935208017

In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.89      0.76      0.82      1436

avg / total       1.00      1.00      1.00    118929



# KNN Classifier

In [68]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn=KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [78]:
y_pred=classifier_knn.predict(X_test)

In [79]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117340,    153],
       [   340,   1096]])

In [80]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8809644962669145

In [81]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117493
          1       0.88      0.76      0.82      1436

avg / total       1.00      1.00      1.00    118929



# Naive Bayes

In [72]:
from sklearn.naive_bayes import GaussianNB
classifier_NB = GaussianNB()
classifier_NB.fit(X_train, y_train)

GaussianNB(priors=None)

In [74]:
y_pred=classifier_NB.predict(X_test)

In [75]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[106608,  10885],
       [     2,   1434]])

In [77]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9529817126306843

In [76]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      0.91      0.95    117493
          1       0.12      1.00      0.21      1436

avg / total       0.99      0.91      0.94    118929

